In [ ]:
# 必要なライブラリをインストール
!pip install -U langchain
!pip install -U openai
!pip install -U llama-index
!pip install -U faiss-cpu


In [ ]:
# OpenAI APIキーの読み込み
import os
os.environ["OPENAI_API_KEY"] = "sk-**********************************************"

In [ ]:
# ログレベルの設定
# デバッグログが不要なら実行しなくて良い
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)

In [ ]:
# インデックスの作成
from llama_index import ServiceContext, GPTVectorStoreIndex, LLMPredictor, StorageContext, load_index_from_storage, JSONReader
from langchain.chat_models import ChatOpenAI

#data dirからテキストファイルの情報を取得
documents = JSONReader().load_data("../data/data_pip/data.json")
# gpt-3.5-turbo を指定（現状デフォルトは davinci ）
service_context = ServiceContext.from_defaults(
  llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name="gpt-3.5-turbo")))
# documents をもとに Embeddings API を通信してベクター取得し GPTVectorStoreIndex を生成
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
# save to disk
index_dir = 'index_pip'
index.storage_context.persist(index_dir)

In [ ]:
# インデックスをファイルから読み込む
storage_context = StorageContext.from_defaults(persist_dir=index_dir)
index = load_index_from_storage(storage_context)

In [ ]:
# テンプレートの作成
from llama_index.prompts.prompts import QuestionAnswerPrompt
QA_PROMPT_TMPL = (
    "私たちは以下の情報をコンテキスト情報として与えます。 \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "質問に対する回答を日本語で最大3件返してください。質問は {query_str} です。\n"
    "なお、それぞれの回答ごとに以下のフォーマットを使用すること。\n"
    "---------------------\n"
    " - タイトル\n"
    " - 回答\n"
    " - スライドのURL\n"
    "---------------------\n"
 )
qa_prompt = QuestionAnswerPrompt(QA_PROMPT_TMPL)

In [ ]:
# プロンプト
question = ""
# ベクター検索 + Chat Completion API 実行
query_engine = index.as_query_engine(
  text_qa_template=qa_prompt
)
response=query_engine.query(question)
print(response)
